In [2]:
import pandas as pd
import glob

In [4]:
# list all files of xlsx in current directory
files = glob.glob('*.xlsx')
files

['provincia_MORONASANTIAGO.xlsx',
 'provincia_NAPO.xlsx',
 'provincia_ORELLANA.xlsx',
 'provincia_PASTAZA.xlsx',
 'provincia_SUCUMBIOS.xlsx',
 'provincia_ZAMORACHINCHIPE.xlsx']

In [13]:
lista = list(range(151))

amazonia_hombres = pd.DataFrame(index=lista)
amazonia_mujeres = pd.DataFrame(index=lista)

for file in files:
    df = pd.read_excel(file)
    # rename P03 to edad 
    df.rename(columns={'P03':'edad'}, inplace=True)
    # make index
    df.set_index(['edad'], inplace=True)
    # drop total column
    df.drop('Total', axis=1, inplace=True)
    # convert to int
    df.index = df.index.astype(int)
    # convert columns to int
    df = df.astype(int)
    # split file name from _ to . 
    nombre = file.split('_')[1]
    nombre = nombre.split('.')[0]
    # rename columns add nombre
    
    amazonia_hombres = amazonia_hombres.join(df['Hombres'], rsuffix=nombre,how='outer')
    amazonia_mujeres = amazonia_mujeres.join(df['Mujeres'], rsuffix=nombre,how='outer')


In [16]:
amazonia_hombres.head()

,Hombres,HombresNAPO,HombresORELLANA,HombresPASTAZA,HombresSUCUMBIOS,HombresZAMORACHINCHIPE,total
0,2194.0,1253.0,1734.0,1056.0,1819.0,1042.0,9098.0
1,2368.0,1445.0,1952.0,1169.0,2157.0,1084.0,10175.0
2,2234.0,1398.0,2000.0,1090.0,2250.0,1199.0,10171.0
3,2240.0,1453.0,2040.0,1029.0,2277.0,1179.0,10218.0
4,2176.0,1352.0,1844.0,1098.0,2202.0,1209.0,9881.0


In [15]:
amazonia_hombres['total'] = amazonia_hombres.sum(axis=1)
amazonia_mujeres['total'] = amazonia_mujeres.sum(axis=1)

In [17]:
# select df where index is less than 99
amazonia_hombres_first = pd.DataFrame(amazonia_hombres[amazonia_hombres.index <= 99])
# select df where index is greater than 99
amazonia_hombres_second = pd.DataFrame(amazonia_hombres[amazonia_hombres.index >= 100])

amazonia_mujeres_first = pd.DataFrame(amazonia_mujeres[amazonia_mujeres.index <= 99])
amazonia_mujeres_second = pd.DataFrame(amazonia_mujeres[amazonia_mujeres.index >= 100])

In [19]:
datos_agrupados_hombres = {}
    # sum by each column
for col in amazonia_hombres_second.columns:
    datos_agrupados_hombres[col] = amazonia_hombres_second[col].sum()

datos_agrupados_mujeres = {}

for col in amazonia_mujeres_second.columns:
    datos_agrupados_mujeres[col] = amazonia_mujeres_second[col].sum()

In [20]:
amazonia_hombres_first.loc[100] = datos_agrupados_hombres
amazonia_mujeres_first.loc[100] = datos_agrupados_mujeres

In [24]:
# fill nan with 0
amazonia_hombres_first.fillna(0, inplace=True)
amazonia_mujeres_first.fillna(0, inplace=True)
# conver to int all columns
amazonia_hombres_first = amazonia_hombres_first.astype(int)
amazonia_mujeres_first = amazonia_mujeres_first.astype(int)

In [25]:
amazonia_hombres_first.head(101)

,Hombres,HombresNAPO,HombresORELLANA,HombresPASTAZA,HombresSUCUMBIOS,HombresZAMORACHINCHIPE,total
0,2194,1253,1734,1056,1819,1042,9098
1,2368,1445,1952,1169,2157,1084,10175
2,2234,1398,2000,1090,2250,1199,10171
3,2240,1453,2040,1029,2277,1179,10218
4,2176,1352,1844,1098,2202,1209,9881
...,...,...,...,...,...,...,...
96,5,4,2,8,9,8,36
97,2,4,1,2,2,4,15
98,2,0,1,1,3,2,9
99,1,0,1,4,4,0,10


In [26]:
# save in one excel file with two sheets
with pd.ExcelWriter('amazonia.xlsx') as writer:
    amazonia_hombres_first.to_excel(writer, sheet_name='Hombres')
    amazonia_mujeres_first.to_excel(writer, sheet_name='Mujeres')
